In [23]:
import pandas as pd
import numpy as np

with open('../datasets/housing/housing.csv') as f:
    data  = pd.read_csv(f)

In [24]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

### Estratificación según ingreso medio

In [25]:
def stratified_data_split(data):
    """ hace un split que refleja la realidad dando iguales proporciones en la muestra sobre un atributo """
    data['income_category'] = np.ceil(data['median_income']/1.5)
    data['income_category'].where(data['income_category'] < 5, 5.0, inplace=True)
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    train_index, test_index = next(split.split(data, data['income_category']))
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]
    strat_test_set.drop(['income_category'], axis=1, inplace=True)
    strat_train_set.drop(['income_category'], axis=1, inplace=True)
    return strat_train_set, strat_test_set


strat_train_set, strat_test_set = stratified_data_split(data)
housing_labels = strat_train_set['median_house_value'].copy()
housing = strat_train_set.drop('median_house_value',axis=1)
# Hago una copia del set de datos para no alterarloo mientras exploro

### reemplazando valores nulos

In [26]:
# relleno los valores nulos con la mediana, pero también hay otras posibildades

from sklearn.preprocessing import Imputer

imputer = Imputer(strategy="median")

# tiro todas las columnas no numericas porque no puedo calcular la mediana en ellas
housing_num = housing.drop('ocean_proximity',axis = 1)
imputer.fit(housing_num)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)

In [27]:
#imputer.statistics_

X = imputer.transform(housing_num)
# X es un array numpy, lo vuelvo a poner en un data frame
housing_tr = pd.DataFrame(X, columns=housing_num.columns)


### encodeando categorias a atributos numericos

In [28]:
# convert text columns to numbers so the algorithms can handle them. Este puede provocar que los algoritmos
#supongan que la categoria 0 sea similar a la 1
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
housing_cat = housing['ocean_proximity']
housing_cat_encoded = encoder.fit_transform(housing_cat)


In [29]:
# para que no haya similitudes falsas, uso un vector binario
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
unspecified = -1
housing_cat_1hot= encoder.fit_transform(housing_cat_encoded.reshape(unspecified,1)) # -1 means unspecified


In [30]:
#To combine the two steps in one we have a LabelBinarizer
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer(sparse_output=True)
housing_cat = housing['ocean_proximity']
housing_cat_encoded_1hot = encoder.fit_transform(housing_cat)


### derivando atributos con mejor correlación

In [31]:
# puedo hacer un Transformer que me agregue atributos, y lo puedo poner luego en un pipeline
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

# Base Estimator me da un get_params y set_params
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
            self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix]/X[:, household_ix]
        population_per_household = X[:, population_ix]/X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix]/X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
attr_adder = CombinedAttributesAdder(False)
housing_extra_attr = attr_adder.transform(housing.values)

### pipeline

In [32]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    """ handles pandas dataframes and returns columns as numpy arrays. There is also a sklearn-pandas egg, and it
    may be that something is added to sklearn in the future as ColumnTransfrmer"""
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [33]:
# combinando todo en pipelines combinadas
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler

# empiezo de nuevo haciendo un split
strat_train_set, strat_test_set = stratified_data_split(data)
housing_labels = strat_train_set['median_house_value'].copy()
housing = strat_train_set.drop('median_house_value',axis=1)


num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([('selector', DataFrameSelector(num_attribs)),
                         ('imputer', Imputer(strategy="median")),
                        ('attribs_adder', CombinedAttributesAdder()),
                        ('std_scaleer', StandardScaler())])
cat_pipeline = Pipeline([('selector', DataFrameSelector(cat_attribs)),
                         ('label_binarizer', LabelBinarizer())])

full_pipeline = FeatureUnion(transformer_list = [('num_pipeline', num_pipeline),
                                                ('cat_pipeline', cat_pipeline)])

housing_prepared = full_pipeline.fit_transform(housing)

### Training a Linear Regression Model

In [34]:
from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression()
linear_reg.fit(housing_prepared, housing_labels)


# empizo de nuevo haciendo un split
strat_train_set, strat_test_set = stratified_data_split(data)
housing_labels = strat_train_set['median_house_value'].copy()
housing = strat_train_set.drop('median_house_value',axis=1)

housing_prepared = full_pipeline.fit_transform(housing)

# entreno en unos pocos ejemplos
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]

some_data_prepared = full_pipeline.transform(some_data)
print("predictions", linear_reg.predict(some_data_prepared))
print("labels", list(some_labels))

predictions [ 210644.60459286  317768.80697211  210956.43331178   59218.98886849
  189747.55849879]
labels [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


#### Mido el RMSE en TODO el training set

In [35]:
from sklearn.metrics import mean_squared_error

housing_predictions = linear_reg.predict(housing_prepared)
linear_mse = mean_squared_error(housing_labels, housing_predictions)
linear_mse = np.sqrt(linear_mse)
print(linear_mse)

68628.1981985


#### como el error es muy significativo, intento con un modelo mas complejo
### DecisionTrees

In [36]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)

0.0


Hice overfitting, intento dividir el training set en kfolds para hacer crossvalidation

In [38]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
print(rmse_scores)

[ 69431.70410223  66132.38846557  70876.994708    69203.76157866
  71178.36699252  74589.12409401  70956.50586076  69888.70946298
  76533.16939319  70430.56168355]


In [41]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
print(forest_rmse)

scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
print(rmse_scores)

22181.3546494
[ 51699.1315119   49841.71870173  52411.67553883  55498.64330241
  51872.48766805  55690.66118272  50894.32585581  50039.44807422
  54675.7246863   53691.02465115]


Una vez que probamos muchos modelos podemos hacer una shortlist con los que mejor funcionan y empezar a tunear los parametros. Para tunear estos parametros hay varios metodos en sklearn

### Sklearn - GridSearchCV:
  Le podemos decir que parametros queremos probar con que valores y prueba todas las combinaciones haciend cross validatoin



In [43]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators':[3,10,30], 'max_features':[2,4,6,8]},
    {'bootstrap':[False], 'n_estimators':[3,10], 'max_features':[2,3,4]}
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(housing_prepared, housing_labels)


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, {'n_estimators': [3, 10], 'bootstrap': [False], 'max_features': [2, 3, 4]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [44]:
grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

In [46]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=8, max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [48]:
cv = grid_search.cv_results_
for mean_score, params in zip(cv["mean_test_score"], cv["params"]):
    print(np.sqrt(-mean_score), params)

64389.8750758 {'n_estimators': 3, 'max_features': 2}
55178.2956624 {'n_estimators': 10, 'max_features': 2}
53033.2969937 {'n_estimators': 30, 'max_features': 2}
60036.9685812 {'n_estimators': 3, 'max_features': 4}
52956.2582577 {'n_estimators': 10, 'max_features': 4}
50590.0671948 {'n_estimators': 30, 'max_features': 4}
59654.6004544 {'n_estimators': 3, 'max_features': 6}
51899.7713668 {'n_estimators': 10, 'max_features': 6}
50289.0612705 {'n_estimators': 30, 'max_features': 6}
58996.36459 {'n_estimators': 3, 'max_features': 8}
51982.5370184 {'n_estimators': 10, 'max_features': 8}
50231.089373 {'n_estimators': 30, 'max_features': 8}
62949.9972417 {'n_estimators': 3, 'bootstrap': False, 'max_features': 2}
54592.3599497 {'n_estimators': 10, 'bootstrap': False, 'max_features': 2}
60698.1964375 {'n_estimators': 3, 'bootstrap': False, 'max_features': 3}
52725.3322369 {'n_estimators': 10, 'bootstrap': False, 'max_features': 3}
58321.7361255 {'n_estimators': 3, 'bootstrap': False, 'max_featur

### Inspeccionar la importancia de los features

In [54]:
extra_attr = ["rooms_per_hhold", "pop_per_hold","bedrooms_per_room"]
cat_one_hot_attr = list(encoder.classes_)
attrs = num_attribs + extra_attr + cat_one_hot_attr

feature_importances = grid_search.best_estimator_.feature_importances_
sorted(zip(feature_importances, attrs), reverse=True)

[(0.409300299919517, 'median_income'),
 (0.14919868990137619, 'INLAND'),
 (0.10826104919133711, 'pop_per_hold'),
 (0.071912115871490198, 'longitude'),
 (0.063917351431586153, 'latitude'),
 (0.046144896734043164, 'rooms_per_hhold'),
 (0.042876887196382572, 'housing_median_age'),
 (0.036219185151234699, 'bedrooms_per_room'),
 (0.015998327350167225, 'total_rooms'),
 (0.015101774128516621, 'total_bedrooms'),
 (0.01481676864951747, 'population'),
 (0.014548196730718801, 'households'),
 (0.0067734409975348129, '<1H OCEAN'),
 (0.0029233522834225122, 'NEAR OCEAN'),
 (0.0019204730085163278, 'NEAR BAY'),
 (8.7191454639277288e-05, 'ISLAND')]

### Evaluar el modelo final en el test set

In [55]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
Y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(Y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print(final_rmse)


47827.1113235
